In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV, LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RepeatedKFold, train_test_split
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier, \
RandomForestRegressor
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import re
from sklearn.preprocessing import StandardScaler
# from skopt import BayesSearchCV
# from skopt.space import Real, Categorical, Integer
# from skopt.plots import plot_objective, plot_histogram, plot_convergence
import warnings
from IPython import display

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier

from xgboost import XGBClassifier, XGBRegressor
from sklearn.inspection import permutation_importance
from lightgbm import LGBMRegressor, LGBMClassifier

In [3]:
train = pd.read_csv('train_regression.csv')
test = pd.read_csv('test_regression.csv')
train.head()

id    host_id  host_since host_location host_response_time  \
0            53872917  366516391  2018-04-09   Chicago, IL     within an hour   
1            39269663   50276775  2014-12-23           NaN     within an hour   
2  556480733030442463  681680651  2020-03-08           NaN     within an hour   
3            52397280   94345291  2015-10-22   Chicago, IL     within an hour   
4  745949202271858960  790998106  2021-04-04   Chicago, IL     within an hour   

  host_response_rate host_acceptance_rate host_is_superhost  \
0                96%                  95%                 f   
1               100%                  97%                 f   
2               100%                 100%                 f   
3               100%                  98%                 f   
4               100%                 100%                 t   

  host_neighbourhood  host_listings_count  ...  review_scores_checkin  \
0  Back of the Yards                   11  ...                   5.00   
1   Clearwater Beach                 1141  ...                    NaN   
2     Lake View East                    1  ...                   5.00   
3           Bucktown                   55  ...                   4.38   
4    Near South Side                   74  ...                   4.83   

  review_scores_communication review_scores_location review_scores_value  \
0                        5.00                   4.28                4.83   
1                         NaN                    NaN                 NaN   
2                        5.00                   4.93                4.80   
3                        4.23                   4.54                3.85   
4                        4.85                   4.74                4.72   

  instant_bookable  calculated_host_listings_count  \
0                f                               9   
1                t                              58   
2                f                               1   
3                t                              55   
4                t                              74   

   calculated_host_listings_count_entire_homes  \
0                                            1   
1                                            0   
2                                            1   
3                                           55   
4                                           74   

  calculated_host_listings_count_private_rooms  \
0                                            8   
1                                           58   
2                                            0   
3                                            0   
4                                            0   

  calculated_host_listings_count_shared_rooms  reviews_per_month  
0                                           0               0.33  
1                                           0                NaN  
2                                           0               2.78  
3                                           0               3.61  
4                                           0               3.24  

[5 rows x 54 columns]

In [4]:
pd.set_option('display.max_columns', None)

train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)

train['host_response_rate'] = train['host_response_rate'].str.rstrip('%').astype(float)
train['host_acceptance_rate'] = train['host_acceptance_rate'].str.rstrip('%').astype(float)
train['host_since'] = pd.to_datetime(train['host_since'])

train['host_is_superhost'] = train['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

train['instant_bookable'] = train['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
train['host_has_profile_pic'] = train['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
train['host_identity_verified'] = train['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
train['has_availability'] = train['has_availability'].apply(lambda x: 1 if x == 't' else 0)


'------------------------------------------------------------------------------------------------'

test['host_response_rate'] = test['host_response_rate'].str.rstrip('%').astype(float)
test['host_acceptance_rate'] = test['host_acceptance_rate'].str.rstrip('%').astype(float)
test['host_since'] = pd.to_datetime(test['host_since'])

test['instant_bookable'] = test['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
test['host_has_profile_pic'] = test['host_has_profile_pic'].apply(lambda x: 1 if x == 't' else 0)
test['host_identity_verified'] = test['host_identity_verified'].apply(lambda x: 1 if x == 't' else 0)
test['has_availability'] = test['has_availability'].apply(lambda x: 1 if x == 't' else 0)


# test['host_is_superhost'] = test['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)


# covert host_since into a usuable form
current_date = pd.to_datetime("today")

train['days_since_host_started'] = (current_date - train['host_since']).dt.days
test['days_since_host_started'] = (current_date - test['host_since']).dt.days

# Impute missing values

#Impute numerical columns with the mean
for column in train.select_dtypes(include=['number']).columns:
    train[column].fillna(train[column].mean(), inplace=True)
    
for column in test.select_dtypes(include=['number']).columns:
    test[column].fillna(train[column].mean(), inplace=True)  # Use train's mean to avoid data leakage

# Impute categorical columns with the mode (most frequent value)
for column in train.select_dtypes(include=['object', 'category']).columns:
    train[column].fillna(train[column].mode()[0], inplace=True)
    
for column in test.select_dtypes(include=['object', 'category']).columns:
    test[column].fillna(train[column].mode()[0], inplace=True)  # Use train's mode to avoid data leakage

# print(test.isnull().any())
# print('---------------------------------------------------------')
# print(train.isnull().any())

def extract_bathrooms(text):
    if 'half' in text.lower():
        return 0.5
    else:
        match = re.search(r'\d+\.?\d*', text)
        return float(match.group()) if match else np.nan

train['bathrooms'] = train['bathrooms_text'].apply(extract_bathrooms)
test['bathrooms'] = test['bathrooms_text'].apply(extract_bathrooms)

train['bathrooms'].fillna(train['bathrooms'].mean(), inplace=True)
test['bathrooms'].fillna(test['bathrooms'].mean(), inplace=True)

train.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)
test.drop(columns=['bathrooms_text', 'host_id', 'host_since', 'first_review', 'last_review'], inplace=True)

train = pd.get_dummies(train)
test = pd.get_dummies(test)

train.head()

#All columns in the dataset, except id, host_id, host_since, first_review, and last_review have
#to be tried as predictors. (price is the response, so it cannot be used as a predictor either.)

C:\Users\benja\AppData\Local\Temp\ipykernel_12736\2113949972.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['price'] = train['price'].str.replace('$', '').str.replace(',', '').astype(float)


id  host_response_rate  host_acceptance_rate  \
0            53872917                96.0                  95.0   
1            39269663               100.0                  97.0   
2  556480733030442463               100.0                 100.0   
3            52397280               100.0                  98.0   
4  745949202271858960               100.0                 100.0   

   host_is_superhost  host_listings_count  host_total_listings_count  \
0                  0                   11                         13   
1                  0                 1141                       2346   
2                  0                    1                          1   
3                  0                   55                         58   
4                  1                   74                         75   

   host_has_profile_pic  host_identity_verified   latitude  longitude  \
0                     1                       1  41.794240 -87.661380   
1                     1                       1  41.892749 -87.620711   
2                     1                       1  41.948160 -87.652309   
3                     1                       1  41.801030 -87.597160   
4                     1                       1  41.855770 -87.624130   

   accommodates  beds   price  minimum_nights  maximum_nights  \
0             1   1.0    30.0              32            1125   
1            12   3.0  1247.0              32             365   
2             6   3.0   214.0               2              45   
3             2   1.0    59.0               2             180   
4             6   3.0   151.0               2             365   

   minimum_minimum_nights  maximum_minimum_nights  minimum_maximum_nights  \
0                      32                      32                    1125   
1                      32                      32                     365   
2                       2                       2                      45   
3                       2                       2                     180   
4                       2                       2                    1125   

   maximum_maximum_nights  minimum_nights_avg_ntm  maximum_nights_avg_ntm  \
0                    1125                    32.0                  1125.0   
1                     365                    32.0                   365.0   
2                      45                     2.0                    45.0   
3                     180                     2.0                   180.0   
4                    1125                     2.0                  1125.0   

   has_availability  availability_30  availability_60  availability_90  \
0                 1               30               60               90   
1                 1               25               49               66   
2                 1                0                0                0   
3                 1               23               53               83   
4                 1                0                0                0   

   availability_365  number_of_reviews  number_of_reviews_ltm  \
0               365                 18                      0   
1               135                  0                      0   
2                 0                 14                     14   
3               355                 13                     13   
4                 0                 64                     42   

   number_of_reviews_l30d  review_scores_rating  review_scores_accuracy  \
0                       0              4.940000                 5.00000   
1                       0              4.759538                 4.79342   
2                       1              4.870000                 5.00000   
3                       3              4.080000                 4.08000   
4                       3              4.800000                 4.74000   

   review_scores_cleanliness  review_scores_checkin  \
0                   4.890000               5.000000   
1                   4.747335  

In [5]:
train_cols = list(train.columns)
test_cols = list(test.columns)

train_cols = [col for col in train_cols if col in test_cols]
test_cols = [col for col in test_cols if col in train_cols]

train_cols == test_cols

True

In [6]:
x_train = train[train_cols].drop(columns=['id'])
y_train = train.price

x_test = test[test_cols].drop(columns=['id'])

def sanitize_column_names(df):
    df.columns = df.columns.str.replace('[', '', regex=True)\
                           .str.replace(']', '', regex=True)\
                           .str.replace('<', '', regex=True)\
                           .str.replace('>', '', regex=True)\
                           .str.replace(' ', '_', regex=True)\
                           .str.replace('[^A-Za-z0-9_]+', '', regex=True)
    return df

x_train = sanitize_column_names(x_train)
x_test = sanitize_column_names(x_test)

In [8]:
model = LGBMRegressor(
                        random_state=1,
                        n_estimators=20000,
                        max_depth=6,
                        subsample=0.75,
                        learning_rate=0.1,
                        
                        reg_lambda=0.1,
                        colsample_bytree=0.8
                     
                     
                     )

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001127 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3715
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 180
[LightGBM] [Info] Start training from score 182.507200
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [9]:
# Narrow with decision tree importance
importances = np.array(model.feature_importances_)
names = np.array(model.feature_name_)

important_features = pd.DataFrame({'feature':names, 'importance':importances})

important_features.sort_values(by='importance', inplace=True, ascending=False)



important_features = list(important_features.head(20).feature)

x_train_reduced = x_train[important_features]
x_test_reduced = x_test[important_features]


model.fit(x_train_reduced, y_train)

y_pred = model.predict(x_test_reduced)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000319 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2430
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 20
[LightGBM] [Info] Start training from score 182.507200
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [30]:
# coarse grid

model = LGBMRegressor(random_state=1)

grid = {
    'n_estimators' : [100, 1000, 10000],
    'max_depth' : [4,6,8],
    'learning_rate' : [0.01, 0.1, 1],
    'subsample' : [0.5, 0.75, 1.0],
    'reg_lambda':[0.01, 0.1, 1], 
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000520 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2430
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 20
[LightGBM] [Info] Start training from score 182.507200
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

In [10]:
# finer grid

model = LGBMRegressor(random_state=1)

grid = {
    'n_estimators' : [50, 100, 500],
    'max_depth' : [6],
    'learning_rate' : [0.001, 0.01, 0.02],
    'subsample' : [0.5, 0.6, 0.7],
    'reg_lambda':[0.5, 1, 10], 
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000473 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2430
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 20
[LightGBM] [Info] Start training from score 182.507200
{'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 50, 'reg_lambda': 10, 'subsample': 0.5}
787.12696420343
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [12]:
# finest grid

model = LGBMRegressor(random_state=1)

grid = {
    'n_estimators' : [50, 70, 80],
    'max_depth' : [4, 6],
    'learning_rate' : [0.005, 0.01, 0.015],
    'subsample' : [0.5, 0.55],
    'reg_lambda':[5, 10, 20, 100], 
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000493 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2430
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 20
[LightGBM] [Info] Start training from score 182.507200
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

In [14]:
# cv settings, shuffle = true

cv_settings = KFold(n_splits=8, shuffle=True, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 8 folds for each of 144 candidates, totalling 1152 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000447 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2430
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 20
[LightGBM] [Info] Start training from score 182.507200
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

In [16]:
# cv settings, n_repeats = 4, n_splits =3

cv_settings = RepeatedKFold(n_splits=3, n_repeats=4, random_state=1)

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 12 folds for each of 144 candidates, totalling 1728 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000363 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2430
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 20
[LightGBM] [Info] Start training from score 182.507200
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

In [22]:
sc = StandardScaler()

sc.fit(x_train_reduced)
x_train_scaled = sc.transform(x_train_reduced)
x_test_scaled = sc.transform(x_test_reduced)

lasso_model = Lasso(alpha=0.005, max_iter=400000)

x_train_subset = x_train_scaled[:500, :]

lasso_model.fit(x_train_subset, y_train[:500])

coefficients = pd.DataFrame({
    'feature': x_train_reduced.columns,
    'importance': np.abs(lasso_model.coef_)
})

coefficients.sort_values(by='importance', ascending=False, inplace=True)

important_features = list(coefficients.head(15).feature)

x_train_reduced = x_train_reduced[important_features]
x_test_reduced = x_test_reduced[important_features]

gscv = GridSearchCV(model, grid, cv=cv_settings, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 12 folds for each of 144 candidates, totalling 1728 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000400 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1787
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 15
[LightGBM] [Info] Start training from score 182.507200
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

In [24]:
# Narrowing features with permutation importance

model = gscv.best_estimator_
model.fit(x_train_reduced, y_train)

perm_importance = permutation_importance(model, x_train_reduced, y_train, n_repeats=10, random_state=1)

feature_names = [col for col in x_train_reduced if col != 'id' and col != 'price']
features_importance = pd.DataFrame({'feature': feature_names, 'importance': perm_importance.importances_mean})

features_importance.sort_values(by='importance', ascending=False, inplace=True)

important_features = list(features_importance.head(10).feature)

x_train_reduced = x_train_reduced[important_features]
x_test_reduced = x_test_reduced[important_features]

model = LGBMRegressor(random_state=1)

grid = {
    'n_estimators' : [100, 1000, 10000],
    'max_depth' : [4,6,8],
    'learning_rate' : [0.01, 0.1, 1],
    'subsample' : [0.5, 0.75, 1.0],
    'reg_lambda':[0.01, 0.1, 1], 
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000322 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1787
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 15
[LightGBM] [Info] Start training from score 182.507200
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] N

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m

In [26]:
model = LGBMRegressor(random_state=1)

grid = {
    'n_estimators' : [50, 100, 500],
    'max_depth' : [6],
    'learning_rate' : [0.001, 0.01, 0.02],
    'subsample' : [0.5, 0.6, 0.7],
    'reg_lambda':[0.5, 1, 10], 
}

gscv = GridSearchCV(model, grid, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1).fit(x_train_reduced, y_train)

print(gscv.best_params_)
print(-gscv.best_score_)

y_pred = gscv.predict(x_test_reduced)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000237 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1243
[LightGBM] [Info] Number of data points in the train set: 5000, number of used features: 10
[LightGBM] [Info] Start training from score 182.507200
{'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 50, 'reg_lambda': 10, 'subsample': 0.5}
788.2083557278597
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [27]:
ids = test.id
results = pd.DataFrame({
    'id': ids,
    'predicted': y_pred
})


results.to_csv('LightGBM_Regression_submission.csv', index=False)
results.head(30)

id   predicted
0   771986218856585018  282.392264
1   855276028675941785  162.764804
2             48537824  201.905823
3             41867473  167.597073
4             28361473  165.523641
5             51911862  138.838836
6   579514121718775234  151.453534
7             46242944  145.466860
8              5571262  161.726442
9   594937541144766707  162.079342
10  772410051557583562  186.525060
11            13165625  150.481180
12            29191606  179.727760
13            32381296  382.744704
14            50451083  167.597073
15            51565135  161.855164
16            47176352  161.783324
17  924507573112993030  145.466860
18            45653951  167.394291
19            26795754  164.924974
20            42035578  158.816920
21            49742536  161.823651
22            48932073  166.246705
23            15904833  176.154084
24  759540854095218350  161.855164
25            29885798  168.158130
26  655444310450112562  227.076931
27            11527162  138.838836
28            18735326  161.726442
29  606007670793701048  189.659463